In [2]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
import statistics

%matplotlib inline

In [33]:
numbers = np.arange(20)

In [34]:
numbers.mean()

9.5

In [35]:
df = pd.DataFrame(numbers, columns=['Number'])

In [36]:
df.rolling(10).mean()

,Number
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,4.5
